In [61]:
import pandas as pd
import numpy as np
import os

def summarize_metrics(path, show_train=True, show_test=True, show_scalar=True, runs=11):
    results = pd.read_pickle(path)

    train_metric_sums = {}
    test_metric_sums = {}
    scalar_sums = {
        'training_time_sec': 0.0,
        'gpu_mem_MB': 0.0,
        'cpu_mem_MB': 0.0,
        'param_count': 0.0
    }

    count = 0
    for run in range(runs):
        if run not in results:
            continue
        entry = results[run]
        count += 1

        for key in scalar_sums:
            scalar_sums[key] += entry.get(key, 0.0)

        for k, v in entry.get('train_metrics', {}).items():
            train_metric_sums[k] = train_metric_sums.get(k, 0.0) + float(v)

        for k, v in entry.get('test_metrics', {}).items():
            test_metric_sums[k] = test_metric_sums.get(k, 0.0) + float(v)

    # 평균 계산
    scalar_avg = {f"avg_{k}": round(v / count, 4) for k, v in scalar_sums.items()} if show_scalar else {}
    train_avg = {f"train_{k}": round(v / count, 4) for k, v in train_metric_sums.items()} if show_train else {}
    test_avg = {f"test_{k}": round(v / count, 4) for k, v in test_metric_sums.items()} if show_test else {}

    # 통합 및 DataFrame 변환
    all_metrics = {**scalar_avg, **train_avg, **test_avg}
    df = pd.DataFrame([all_metrics])

    return df

dataset_order = [
    "basic", "gaussian", "uniform", "outlier", "edge", "BA", "ER", "grid", "tree",
    "education", "election", "income", "unemployment",
    "PTBR", "chameleon", "crocodile", "squirrel", "Anaheim", "ChicagoSketch"
]

def get_dataset_index(filename):
    for i, name in enumerate(dataset_order):
        if name in filename:
            return i
    return len(dataset_order)  # 못 찾은 건 맨 뒤로

def summarize_model_resources(path, runs=11):
    results = pd.read_pickle(path)

    models = ['gnn', 'conf_gnn']
    scalar_keys = ['training_time_sec', 'gpu_mem_MB', 'cpu_mem_MB', 'param_count']

    # 모델별 누적 딕셔너리 초기화
    model_sums = {model: {k: 0.0 for k in scalar_keys} for model in models}
    model_counts = {model: 0 for model in models}

    for run in range(runs):
        if run not in results:
            continue
        for model in models:
            entry = results[run].get(model, {})
            if not entry:
                continue
            model_counts[model] += 1
            for key in scalar_keys:
                model_sums[model][key] += entry.get(key, 0.0)

    # 평균 계산
    model_avgs = {}
    for model in models:
        if model_counts[model] == 0:
            continue
        avg_metrics = {
            f"{model}_{k}": round(v / model_counts[model], 4)
            for k, v in model_sums[model].items()
        }
        model_avgs.update(avg_metrics)

    df = pd.DataFrame([model_avgs])
    return df

### SQR

In [53]:
train, test, scalar = False, False, True

folder_path = "/home/sypark/GQNN/pred/SQR"
pkl_files = [f for f in os.listdir(folder_path) if f.endswith('.pkl')]
sorted_pkl_files = sorted(pkl_files, key=get_dataset_index)

for i in range(len(pkl_files)):
    print('SQR', sorted_pkl_files[i], 'Results...')
    path = os.path.join(folder_path, sorted_pkl_files[i])
    print(summarize_metrics(path=path, show_train=train, show_test=test, show_scalar=scalar).T) 
    print('-' * 30)

SQR basic_SQR.pkl Results...
                              0
avg_training_time_sec     1.800
avg_gpu_mem_MB           65.789
avg_cpu_mem_MB         1231.970
avg_param_count        9153.000
------------------------------
SQR gaussian_SQR.pkl Results...
                              0
avg_training_time_sec     1.887
avg_gpu_mem_MB           65.789
avg_cpu_mem_MB         1230.541
avg_param_count        9153.000
------------------------------
SQR uniform_SQR.pkl Results...
                              0
avg_training_time_sec     1.889
avg_gpu_mem_MB           65.789
avg_cpu_mem_MB         1232.142
avg_param_count        9153.000
------------------------------
SQR outlier_SQR.pkl Results...
                              0
avg_training_time_sec     1.978
avg_gpu_mem_MB           65.789
avg_cpu_mem_MB         1232.871
avg_param_count        9153.000
------------------------------
SQR edge_SQR.pkl Results...
                              0
avg_training_time_sec     1.937
avg_gpu_mem_MB       

### RQR

In [54]:
train, test, scalar = False, False, True

folder_path = "/home/sypark/GQNN/pred/RQR"
pkl_files = [f for f in os.listdir(folder_path) if f.endswith('.pkl')]
sorted_pkl_files = sorted(pkl_files, key=get_dataset_index)

for i in range(len(pkl_files)):
    print('RQR', sorted_pkl_files[i], 'Results...')
    path = os.path.join(folder_path, sorted_pkl_files[i])
    print(summarize_metrics(path=path, show_train=train, show_test=test, show_scalar=scalar).T) 
    print('-' * 30)

RQR basic_RQR.pkl Results...
                              0
avg_training_time_sec     2.068
avg_gpu_mem_MB           85.999
avg_cpu_mem_MB         1294.955
avg_param_count        9090.000
------------------------------
RQR gaussian_RQR.pkl Results...
                              0
avg_training_time_sec     1.913
avg_gpu_mem_MB           85.999
avg_cpu_mem_MB         1295.988
avg_param_count        9090.000
------------------------------
RQR uniform_RQR.pkl Results...
                              0
avg_training_time_sec     1.900
avg_gpu_mem_MB           85.999
avg_cpu_mem_MB         1295.865
avg_param_count        9090.000
------------------------------
RQR outlier_RQR.pkl Results...
                              0
avg_training_time_sec     2.041
avg_gpu_mem_MB           85.999
avg_cpu_mem_MB         1295.324
avg_param_count        9090.000
------------------------------
RQR edge_RQR.pkl Results...
                              0
avg_training_time_sec     1.912
avg_gpu_mem_MB       

### BNN

In [55]:
train, test, scalar = False, False, True

folder_path = "/home/sypark/GQNN/pred/BNN"
pkl_files = [f for f in os.listdir(folder_path) if f.endswith('.pkl')]
sorted_pkl_files = sorted(pkl_files, key=get_dataset_index)

for i in range(len(pkl_files)):
    print('BNN', sorted_pkl_files[i], 'Results...')
    path = os.path.join(folder_path, sorted_pkl_files[i])
    print(summarize_metrics(path=path, show_train=train, show_test=test, show_scalar=scalar).T) 
    print('-' * 30)

BNN basic_BNN.pkl Results...
                              0
avg_training_time_sec     1.693
avg_gpu_mem_MB           65.769
avg_cpu_mem_MB         1263.186
avg_param_count        9090.000
------------------------------
BNN gaussian_BNN.pkl Results...
                              0
avg_training_time_sec     1.560
avg_gpu_mem_MB           65.769
avg_cpu_mem_MB         1263.401
avg_param_count        9090.000
------------------------------
BNN uniform_BNN.pkl Results...
                              0
avg_training_time_sec     1.713
avg_gpu_mem_MB           65.769
avg_cpu_mem_MB         1262.642
avg_param_count        9090.000
------------------------------
BNN outlier_BNN.pkl Results...
                              0
avg_training_time_sec     1.432
avg_gpu_mem_MB           65.769
avg_cpu_mem_MB         1263.419
avg_param_count        9090.000
------------------------------
BNN edge_BNN.pkl Results...
                              0
avg_training_time_sec     1.453
avg_gpu_mem_MB       

### MC

In [56]:
train, test, scalar = False, False, True

folder_path = "/home/sypark/GQNN/pred/MC"
pkl_files = [f for f in os.listdir(folder_path) if f.endswith('.pkl')]
sorted_pkl_files = sorted(pkl_files, key=get_dataset_index)

for i in range(len(pkl_files)):
    print('MC', sorted_pkl_files[i], 'Results...')
    path = os.path.join(folder_path, sorted_pkl_files[i])
    print(summarize_metrics(path=path, show_train=train, show_test=test, show_scalar=scalar).T) 
    print('-' * 30)

MC basic_MC.pkl Results...
                              0
avg_training_time_sec     1.428
avg_gpu_mem_MB           66.009
avg_cpu_mem_MB         1237.782
avg_param_count        9025.000
------------------------------
MC gaussian_MC.pkl Results...
                              0
avg_training_time_sec     1.461
avg_gpu_mem_MB           66.009
avg_cpu_mem_MB         1237.172
avg_param_count        9025.000
------------------------------
MC uniform_MC.pkl Results...
                              0
avg_training_time_sec     1.460
avg_gpu_mem_MB           66.009
avg_cpu_mem_MB         1236.890
avg_param_count        9025.000
------------------------------
MC outlier_MC.pkl Results...
                              0
avg_training_time_sec     1.465
avg_gpu_mem_MB           66.009
avg_cpu_mem_MB         1236.305
avg_param_count        9025.000
------------------------------
MC edge_MC.pkl Results...
                              0
avg_training_time_sec     1.441
avg_gpu_mem_MB           66.009

### CP

In [57]:
train, test, scalar = False, False, True

folder_path = "/home/sypark/GQNN/pred/CP"
pkl_files = [f for f in os.listdir(folder_path) if f.endswith('.pkl')]
sorted_pkl_files = sorted(pkl_files, key=get_dataset_index)

for i in range(len(pkl_files)):
    print('CP', sorted_pkl_files[i], 'Results...')
    path = os.path.join(folder_path, sorted_pkl_files[i])
    print(summarize_metrics(path=path, show_train=train, show_test=test, show_scalar=scalar).T) 
    print('-' * 30)

CP basic_CP.pkl Results...
                              0
avg_training_time_sec     1.372
avg_gpu_mem_MB           65.678
avg_cpu_mem_MB         1229.489
avg_param_count        9025.000
------------------------------
CP gaussian_CP.pkl Results...
                              0
avg_training_time_sec     1.549
avg_gpu_mem_MB           65.678
avg_cpu_mem_MB         1229.168
avg_param_count        9025.000
------------------------------
CP uniform_CP.pkl Results...
                              0
avg_training_time_sec     1.597
avg_gpu_mem_MB           65.678
avg_cpu_mem_MB         1230.165
avg_param_count        9025.000
------------------------------
CP outlier_CP.pkl Results...
                              0
avg_training_time_sec     1.705
avg_gpu_mem_MB           65.678
avg_cpu_mem_MB         1229.977
avg_param_count        9025.000
------------------------------
CP edge_CP.pkl Results...
                              0
avg_training_time_sec     1.728
avg_gpu_mem_MB           65.678

### CF-GNN

In [62]:
train, test, scalar = True, True, False

folder_path = "/home/sypark/GQNN/pred/CF-GNN"
pkl_files = [f for f in os.listdir(folder_path) if f.endswith('.pkl')]
sorted_pkl_files = sorted(pkl_files, key=get_dataset_index)

for i in range(len(pkl_files)):
    print('CF-GNN', sorted_pkl_files[i], 'Results...')
    path = os.path.join(folder_path, sorted_pkl_files[i])
    print(summarize_metrics(path=path, show_train=train, show_test=test, show_scalar=scalar).T) 
    print('-' * 30)

CF-GNN basic_GraphSAGE_confgnn.pkl Results...
                      0
train_PCIP       0.9252
train_MPIW       1.9024
train_NMPIW      1.8114
train_MCT        0.0458
train_CWC        3.3521
train_MPE        0.5818
train_Sharpness  4.6370
train_WS         1.9071
test_PCIP        0.9220
test_MPIW        1.9014
test_NMPIW       1.7983
test_MCT         0.0537
test_CWC         3.2559
test_MPE         0.5702
test_Sharpness   4.7865
test_WS          1.9070
------------------------------
CF-GNN gaussian_GraphSAGE_confgnn.pkl Results...
                      0
train_PCIP       0.8880
train_MPIW       2.7293
train_NMPIW      1.2703
train_MCT        0.0551
train_CWC        2.7611
train_MPE        0.7338
train_Sharpness  8.4883
train_WS         2.7560
test_PCIP        0.9105
test_MPIW        2.9003
test_NMPIW       1.3057
test_MCT         0.0403
test_CWC         2.4958
test_MPE         0.7432
test_Sharpness   9.4711
test_WS          2.9217
------------------------------
CF-GNN uniform_GraphSAGE_co

In [63]:
for i in range(len(pkl_files)):
    print('CF-GNN', sorted_pkl_files[i], 'Results...')
    path = os.path.join(folder_path, sorted_pkl_files[i])
    print(summarize_model_resources(path=path).T) 
    print('-' * 30)
summarize_model_resources

CF-GNN basic_GraphSAGE_confgnn.pkl Results...
                                   0
gnn_training_time_sec          3.168
gnn_gpu_mem_MB                66.067
gnn_cpu_mem_MB              1507.779
gnn_param_count             1091.000
conf_gnn_training_time_sec     8.297
conf_gnn_gpu_mem_MB           65.730
conf_gnn_cpu_mem_MB         1517.417
conf_gnn_param_count        1091.000
------------------------------
CF-GNN gaussian_GraphSAGE_confgnn.pkl Results...
                                   0
gnn_training_time_sec          3.090
gnn_gpu_mem_MB                66.067
gnn_cpu_mem_MB              1506.917
gnn_param_count             1091.000
conf_gnn_training_time_sec     8.074
conf_gnn_gpu_mem_MB           65.730
conf_gnn_cpu_mem_MB         1516.563
conf_gnn_param_count        1091.000
------------------------------
CF-GNN uniform_GraphSAGE_confgnn.pkl Results...
                                   0
gnn_training_time_sec          3.125
gnn_gpu_mem_MB                66.067
gnn_cpu_mem_MB    

<function __main__.summarize_model_resources(path, runs=11)>

### GQNN

In [64]:
train, test, scalar = False, False, True

folder_path = "/home/sypark/GQNN/pred/GQNN"
pkl_files = [f for f in os.listdir(folder_path) if f.endswith('.pkl')]
sorted_pkl_files = sorted(pkl_files, key=get_dataset_index)

for i in range(len(pkl_files)):
    print('GQNN', sorted_pkl_files[i], 'Results...')
    path = os.path.join(folder_path, sorted_pkl_files[i])
    print(summarize_metrics(path=path, show_train=train, show_test=test, show_scalar=scalar).T) 
    print('-' * 30)

GQNN basic_GQNN_lf(0.05).pkl Results...
                              0
avg_training_time_sec     2.215
avg_gpu_mem_MB           65.770
avg_cpu_mem_MB         1295.027
avg_param_count        9090.000
------------------------------
GQNN gaussian_GQNN_lf(0.05).pkl Results...
                              0
avg_training_time_sec     2.203
avg_gpu_mem_MB           65.770
avg_cpu_mem_MB         1295.854
avg_param_count        9090.000
------------------------------
GQNN uniform_GQNN_lf(0.05).pkl Results...
                              0
avg_training_time_sec     2.168
avg_gpu_mem_MB           65.770
avg_cpu_mem_MB         1294.289
avg_param_count        9090.000
------------------------------
GQNN outlier_GQNN_lf(0.05).pkl Results...
                              0
avg_training_time_sec     2.206
avg_gpu_mem_MB           65.770
avg_cpu_mem_MB         1294.511
avg_param_count        9090.000
------------------------------
GQNN edge_GQNN_lf(0.05).pkl Results...
                              